# Continuous Optimization

In [1]:
# This code cell installs packages on Colab

import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/optimization/main/notebooks/helper.py"
    import helper
    helper.easy_install()
else:
    sys.path.insert(0, '../')
    import helper
helper.set_plotting_style()

In [12]:
import pandas as pd
import pyomo.environ as pyo
import idaes

## Linear Programs: Student Diet Example

Reference: https://docs.mosek.com/modeling-cookbook/linear.html

You want to save money eating while remaining healthy. A healthy diet requires at least P=6 units of protein, C=15 units of carbohydrates, F=5 units of fats and V=7 units of vitamins. Due to compounding factors (blizzard during Lent), our campus only has these options:

In [3]:
# Load data from file, use the first column (0, recall Python starts counting at 0) as the index
food_options = pd.read_csv('../data/student_diet.csv',index_col=0)

# Print up the the first 10 rows of data
food_options.head(10)

,P,C,F,V,price
takeaway,3.0,3,2,1,5
vegetables,1.0,2,0,4,1
bread,0.5,4,1,0,2


Let's build a Python dictionary to store the nutrient requirements. (I strongly recommend not touching Python until we write the model on paper. I am including this here to avoid scrolling between the problem description and this cell.)

In [4]:
# Uncomment and fill in with all of the data
# nutrient_requirements = {'P':6, 'C':15 }

### BEGIN SOLUTION
nutrient_requirements = {'P':6, 'C':15, 'F':5, 'V':7}
### END SOLUTION

### Propose an Optimization Model

**Sets**

**Parameters**

**Variables**

**Constraints**

**Degree of Freedom Analysis**

We will later learn more about how to factor inequality constraints into degree of freedom analysis. For now, please count the number of equality and inequality constraints separately.

### Solve in Pyomo

With our optimization model written on paper, we can proceed to solve in Pyomo. Before we start, let's review a few pandas tricks.

In [5]:
# Extract the column names, convert to a list
food_options.columns.to_list()

['P', 'C', 'F', 'V', 'price']

In [6]:
# Same as above, but drop the last entry
nutrients = food_options.columns.to_list()[0:4]
nutrients

['P', 'C', 'F', 'V']

In [7]:
# Extract the index names, convert to a list
foods = food_options.index.to_list()
foods

['takeaway', 'vegetables', 'bread']

In [8]:
# Create a dictionary with keys such as ('takeaway', 'P')
# Do not include 'price'
food_info = food_options[nutrients].stack().to_dict()
food_info

{('takeaway', 'P'): 3.0,
 ('takeaway', 'C'): 3.0,
 ('takeaway', 'F'): 2.0,
 ('takeaway', 'V'): 1.0,
 ('vegetables', 'P'): 1.0,
 ('vegetables', 'C'): 2.0,
 ('vegetables', 'F'): 0.0,
 ('vegetables', 'V'): 4.0,
 ('bread', 'P'): 0.5,
 ('bread', 'C'): 4.0,
 ('bread', 'F'): 1.0,
 ('bread', 'V'): 0.0}

In [9]:
# Create dictionary of only prices
price = food_options['price'].to_dict()
price

{'takeaway': 5, 'vegetables': 1, 'bread': 2}

Now let's build our Pyomo model!

In [10]:
### BEGIN SOLUTION

m = pyo.ConcreteModel()

## Define sets
m.FOOD = pyo.Set(initialize = foods)
m.NUTRIENTS = pyo.Set(initialize = nutrients)

## Define parameters
m.needs = pyo.Param(m.NUTRIENTS, initialize=nutrient_requirements)
m.food_info = pyo.Param(m.FOOD, m.NUTRIENTS, initialize=food_info)
m.price = pyo.Param(m.FOOD, initialize=price)

## Define variables
m.food_eaten = pyo.Var(m.FOOD,initialize=1.0,domain=pyo.NonNegativeReals)

## Define constraints
def diet_min_rule(m, n):
    return sum(m.food_info[f,n] * m.food_eaten[f] for f in m.FOOD) >= m.needs[n]

m.diet_min = pyo.Constraint(m.NUTRIENTS, rule=diet_min_rule)

## Define objective
m.cost = pyo.Objective(expr=sum(m.food_eaten[f]*m.price[f] for f in m.FOOD), sense=pyo.minimize)

# Print model
m.pprint()

### END SOLUTION

3 Set Declarations
    FOOD : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'takeaway', 'vegetables', 'bread'}
    NUTRIENTS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'P', 'C', 'F', 'V'}
    food_info_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain         : Size : Members
        None :     2 : FOOD*NUTRIENTS :   12 : {('takeaway', 'P'), ('takeaway', 'C'), ('takeaway', 'F'), ('takeaway', 'V'), ('vegetables', 'P'), ('vegetables', 'C'), ('vegetables', 'F'), ('vegetables', 'V'), ('bread', 'P'), ('bread', 'C'), ('bread', 'F'), ('bread', 'V')}

3 Param Declarations
    food_info : Size=12, Index=food_info_index, Domain=Any, Default=None, Mutable=False
        Key                 : Value
             ('bread', 'C') :   4.0
             ('bread', 'F') :   1.0
             ('bread', 'P') :   0.5
             

```{dropdown} Click to see the solution to the activity

```python
m = pyo.ConcreteModel()

## Define sets
m.FOOD = pyo.Set(initialize = foods)
m.NUTRIENTS = pyo.Set(initialize = nutrients)

## Define parameters
m.needs = pyo.Param(m.NUTRIENTS, initialize=nutrient_requirements)
m.food_info = pyo.Param(m.FOOD, m.NUTRIENTS, initialize=food_info)
m.price = pyo.Param(m.FOOD, initialize=price)

## Define variables
m.food_eaten = pyo.Var(m.FOOD,initialize=1.0,domain=pyo.NonNegativeReals)

## Define constraints
def diet_min_rule(m, n):
    return sum(m.food_info[f,n] * m.food_eaten[f] for f in m.FOOD) >= m.needs[n]

m.diet_min = pyo.Constraint(m.NUTRIENTS, rule=diet_min_rule)

## Define objective
m.cost = pyo.Objective(expr=sum(m.food_eaten[f]*m.price[f] for f in m.FOOD), sense=pyo.minimize)

# Print model
m.pprint()

<div style="background-color: rgba(0,0,255,0.05) ; padding: 10px; border: 1px solid darkblue;"> 
<b>Activity</b>: Check the Pyomo model. Specifically, are the input (parameter) data correct? Do the equations match our model written on paper?
</div>

In [13]:
# Specify the solver


solver = pyo.SolverFactory('ipopt')

# Solve
results = solver.solve(m, tee=True)

ipopt


ApplicationError: No executable found for solver 'ipopt'

<div style="background-color: rgba(0,0,255,0.05) ; padding: 10px; border: 1px solid darkblue;"> 
<b>Activity</b>: Does your degree of freedom analysis match Ipopt?
</div>


```{dropdown} Solution
Add Solution HERE
```


### Analyze Results

Let's extract the solution.

In [12]:
### BEGIN SOLUTION
for f in m.FOOD:
    print("Units of",f,"eaten =",m.food_eaten[f]())
### END SOLUTION

Units of takeaway eaten = 0.9999999904584188
Units of vegetables eaten = 1.4999999892477955
Units of bread eaten = 2.9999999704842386


Extract the KKT Multipliers

In [ ]:
#Inspect dual variables for lower bound
m.ipopt_zL_out.display()

In [ ]:
#Inspect dual variables for upper bound
m.ipopt_zU_out.display()

TODO: After we discuss optimization theory, add discussion of shadow prices and multipliers here.

## Take Away Messages - Update for just LP
* Linear programs are convex. We will learn this means all local optima are global optima.
* Nonlinear programs may be nonconvex. For nonconvex problems, there often existings many local optima that are not also global optima.
* We will learn how to mathematically define convexity and analyze this property.
* Initialization is really important in optimization problems with nonlinear objectives or constraints!
* There are specialize solves for linear programs, quadratic programs, and convex programs. In this class, we will focus on more general algorithms for (non)convex nonlinear programs including the algorithms used by the `ipopt` solver.